# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import pickle
import sqlalchemy
import numpy

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download(['punkt', 'stopwords', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /home/zfsdt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/zfsdt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/zfsdt/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/zfsdt/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# load data from database
engine = sqlalchemy.create_engine('sqlite:///messages.db')
df = pd.read_sql_table('cleandata', engine)
# ignoring 'original' text because translation is available in message
# We can use 'genre' for model improvement later
X = df[['message', 'genre']]
# child_alone is always 0 in our data set
y = df.drop(columns=['id', 'message', 'original', 'genre', 'child_alone'])

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X['message'], y, test_size=.2, random_state=42)

In [6]:
pipeline.fit(X_train, y_train)

/usr/lib/python3/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/lib/python3/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/lib/python3/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/lib/python3/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/lib/python3/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change 

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def report(y_test, y_pred):
    y_pred.columns = y_test.columns
    for column in y.columns:
        print('** {} **'.format(column).upper())
        print(classification_report(y_test[column], y_pred[column]))

y_pred = pd.DataFrame(pipeline.predict(X_test))
report(y_test, y_pred)

** RELATED **
              precision    recall  f1-score   support

           0       0.61      0.35      0.45      1245
           1       0.82      0.93      0.87      3998

   micro avg       0.79      0.79      0.79      5243
   macro avg       0.71      0.64      0.66      5243
weighted avg       0.77      0.79      0.77      5243

** REQUEST **
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      4352
           1       0.82      0.38      0.52       891

   micro avg       0.88      0.88      0.88      5243
   macro avg       0.85      0.68      0.73      5243
weighted avg       0.87      0.88      0.86      5243

** OFFER **
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5219
           1       0.00      0.00      0.00        24

   micro avg       1.00      1.00      1.00      5243
   macro avg       0.50      0.50      0.50      5243
weighted avg       0.99      1.00 

/usr/lib/python3/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.96      1.00      0.98      5008
           1       0.69      0.08      0.14       235

   micro avg       0.96      0.96      0.96      5243
   macro avg       0.83      0.54      0.56      5243
weighted avg       0.95      0.96      0.94      5243

** BUILDINGS **
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      4974
           1       0.67      0.06      0.11       269

   micro avg       0.95      0.95      0.95      5243
   macro avg       0.81      0.53      0.54      5243
weighted avg       0.94      0.95      0.93      5243

** ELECTRICITY **
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5128
           1       1.00      0.04      0.08       115

   micro avg       0.98      0.98      0.98      5243
   macro avg       0.99      0.52      0.54      5243
weighted avg       0.98      0.98      0

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
# Below we will be using f1_macro for the grid search
f1_score(y_test, y_pred, average='macro')

/usr/lib/python3/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.21644214078898866

In [9]:
# Get initial parameters
# Below we will be using f1_macro for the grid search
pipeline.get_params()['clf']

MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None)

In [10]:
parameters = {
    'clf__estimator__n_estimators' : (10, 20, 40),
    'clf__estimator__min_samples_split' : (2, 5, 10)
}   

# Vincent van Asch: Macro- and micro-averaged evaluation measures, 2013, Draft
# https://pdfs.semanticscholar.org/1d10/6a2730801b6210a67f7622e4d192bb309303.pdf
# suggests using a macro measure
cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_macro', verbose=10, n_jobs=4, pre_dispatch=4)

cv.fit(X_train, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


/usr/lib/python3/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:  2.4min
/usr/lib/python3/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:  6.2min
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:  8.7min
[Parallel(n_jobs=4)]: Done  21 tasks      | elapsed: 12.1min
[Parallel(n_jobs=4)]: Done  27 out of  27 | elapsed: 16.3min remaining:    0.0s
[Parall

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'clf__estimator__n_estimators': (10, 20, 40), 'clf__estimator__min_samples_split': (2, 5, 10)},
       pre_dispatch=4, refit=True, return_train_score='warn',
       scoring='f1_macro', verbose=10)

In [11]:
cv.best_params_

{'clf__estimator__min_samples_split': 5, 'clf__estimator__n_estimators': 10}

In [12]:
cv.cv_results_

/usr/lib/python3/dist-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/lib/python3/dist-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/lib/python3/dist-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/lib/python3/dist-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are acc

{'mean_fit_time': array([ 59.66358844, 123.24644033, 216.39975301,  49.38463521,
         90.45888845, 171.6136415 ,  45.34258493,  78.62943045,
        144.69555847]),
 'std_fit_time': array([0.27837459, 6.27205464, 7.53688599, 2.56897027, 3.4683071 ,
        0.96682865, 0.95421014, 1.57009037, 0.28341452]),
 'mean_score_time': array([ 6.11949499, 10.4165868 , 11.79609625,  5.90525031,  8.76551636,
        11.71411109,  5.89199082,  8.05368932, 11.4232889 ]),
 'std_score_time': array([0.15468437, 0.69371239, 0.31336105, 0.02534963, 0.94236754,
        0.29972325, 0.09595398, 0.39792155, 0.16111539]),
 'param_clf__estimator__min_samples_split': masked_array(data=[2, 2, 2, 5, 5, 5, 10, 10, 10],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_clf__estimator__n_estimators': masked_array(data=[10, 20, 40, 10, 20, 40, 10, 20, 40],
              mask=[False, False, False, False

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
pipeline.set_params(**cv.best_params_)
pipeline.fit(X_train, y_train)
y_pred = pd.DataFrame(pipeline.predict(X_test))
f1_score(y_test, y_pred, average='macro')

/usr/lib/python3/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.2444339896525875

In [14]:
report(y_test, y_pred)

/usr/lib/python3/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


** RELATED **
              precision    recall  f1-score   support

           0       0.67      0.29      0.40      1245
           1       0.81      0.96      0.88      3998

   micro avg       0.80      0.80      0.80      5243
   macro avg       0.74      0.62      0.64      5243
weighted avg       0.78      0.80      0.76      5243

** REQUEST **
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4352
           1       0.84      0.45      0.58       891

   micro avg       0.89      0.89      0.89      5243
   macro avg       0.87      0.71      0.76      5243
weighted avg       0.89      0.89      0.88      5243

** OFFER **
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5219
           1       0.00      0.00      0.00        24

   micro avg       1.00      1.00      1.00      5243
   macro avg       0.50      0.50      0.50      5243
weighted avg       0.99      1.00 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### Try SVC instead of random forest

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X['message'], y, test_size=.2, random_state=42)
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(SVC()))
])

pipeline2.fit(X_train, y_train)
y_pred = pipeline2.predict(X_test)
f1_score(y_test, y_pred, average='macro')

/usr/lib/python3/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/lib/python3/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/lib/python3/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/lib/python3/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 

/usr/lib/python3/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/lib/python3/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/lib/python3/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/lib/python3/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 

0.024722123456026715

In [16]:
parameters2 = [{'clf__estimator__kernel': ['rbf'],
                'clf__estimator__gamma': [1e-3, 1e-4],
                'clf__estimator__C': [1, 10, 100, 1000]},
               {'clf__estimator__kernel': ['linear'],
                'clf__estimator__C': [1, 10, 100, 1000]}]

cv2 = GridSearchCV(pipeline2, param_grid=parameters2, scoring='f1_macro', verbose=10, n_jobs=4, pre_dispatch=4)

cv2.fit(X_train, y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


/usr/lib/python3/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed: 15.7min
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed: 18.5min
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed: 51.3min
/usr/lib/python3/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed: 77.5min
[Parallel(n_jobs=4)]: Done  21 tasks      | elapsed: 120.4min
[Parallel(n_jobs=4)]: Done  28 tasks      | elapsed: 158.5min
[Parallel(n_jobs=4)]: Do

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ity=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid=[{'clf__estimator__kernel': ['rbf'], 'clf__estimator__gamma': [0.001, 0.0001], 'clf__estimator__C': [1, 10, 100, 1000]}, {'clf__estimator__kernel': ['linear'], 'clf__estimator__C': [1, 10, 100, 1000]}],
       pre_dispatch=4, refit=True, return_train_score='warn',
       scoring='f1_macro', verbose=10)

In [17]:
#X_train, X_test, y_train, y_test = train_test_split(X['message'], y, test_size=.2, random_state=42)
pipeline2.set_params(**cv2.best_params_)
pipeline2.fit(X_train, y_train)
y_pred = pd.DataFrame(pipeline2.predict(X_test))
f1_score(y_test, y_pred, average='macro')

/usr/lib/python3/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.4200254801270426

In [18]:
cv2.best_params_

{'clf__estimator__C': 1000,
 'clf__estimator__gamma': 0.001,
 'clf__estimator__kernel': 'rbf'}

In [19]:
report(y_test, y_pred)

/usr/lib/python3/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


** RELATED **
              precision    recall  f1-score   support

           0       0.66      0.55      0.60      1245
           1       0.87      0.91      0.89      3998

   micro avg       0.83      0.83      0.83      5243
   macro avg       0.76      0.73      0.74      5243
weighted avg       0.82      0.83      0.82      5243

** REQUEST **
              precision    recall  f1-score   support

           0       0.92      0.97      0.94      4352
           1       0.79      0.61      0.69       891

   micro avg       0.91      0.91      0.91      5243
   macro avg       0.86      0.79      0.82      5243
weighted avg       0.90      0.91      0.90      5243

** OFFER **
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5219
           1       0.00      0.00      0.00        24

   micro avg       1.00      1.00      1.00      5243
   macro avg       0.50      0.50      0.50      5243
weighted avg       0.99      1.00 

              precision    recall  f1-score   support

           0       0.89      0.95      0.92      4233
           1       0.70      0.51      0.59      1010

   micro avg       0.86      0.86      0.86      5243
   macro avg       0.79      0.73      0.75      5243
weighted avg       0.85      0.86      0.85      5243



### Include column 'genre'

In [20]:
ct = ColumnTransformer(transformers = [
        ('msg', Pipeline(steps = [
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
            ]), 'message'),
        ('gnr', OneHotEncoder(), ['genre'] )
    ])

In [21]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=.2, random_state=42)

In [22]:
pipeline3 = Pipeline([
    ('ct', ct),
    ('clf', MultiOutputClassifier(SVC()))
])

In [23]:
pipeline3.set_params(**cv2.best_params_)
pipeline3.fit(X_train2, y_train2)

Pipeline(memory=None,
     steps=[('ct', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('msg', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>,...ity=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
           n_jobs=None))])

In [24]:
y_pred2 = pipeline3.predict(X_test2)
f1_score(y_test2, y_pred2, average='macro')

/usr/lib/python3/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.41837717926247253

In [28]:
y_pred2 = pd.DataFrame(y_pred2)
report(y_test2, y_pred2)

** RELATED **
              precision    recall  f1-score   support

           0       0.66      0.55      0.60      1245
           1       0.87      0.91      0.89      3998

   micro avg       0.83      0.83      0.83      5243
   macro avg       0.76      0.73      0.74      5243
weighted avg       0.82      0.83      0.82      5243

** REQUEST **
              precision    recall  f1-score   support

           0       0.92      0.97      0.94      4352
           1       0.78      0.61      0.69       891

   micro avg       0.91      0.91      0.91      5243
   macro avg       0.85      0.79      0.81      5243
weighted avg       0.90      0.91      0.90      5243

** OFFER **
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5219
           1       0.00      0.00      0.00        24

   micro avg       1.00      1.00      1.00      5243
   macro avg       0.50      0.50      0.50      5243
weighted avg       0.99      1.00 

/usr/lib/python3/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.94      0.99      0.97      4907
           1       0.42      0.06      0.10       336

   micro avg       0.93      0.93      0.93      5243
   macro avg       0.68      0.53      0.53      5243
weighted avg       0.91      0.93      0.91      5243

** TRANSPORT **
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      5008
           1       0.73      0.20      0.31       235

   micro avg       0.96      0.96      0.96      5243
   macro avg       0.85      0.60      0.65      5243
weighted avg       0.95      0.96      0.95      5243

** BUILDINGS **
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      4974
           1       0.71      0.38      0.50       269

   micro avg       0.96      0.96      0.96      5243
   macro avg       0.84      0.69      0.74      5243
weighted avg       0.95      0.96      0.9

### 9. Export your model as a pickle file

see https://scikit-learn.org/stable/modules/model_persistence.html

In [30]:
outfile = open('pipeline.pickle','wb')
pickle.dump(pipeline3, outfile)
outfile.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.